--------------------------------------------------------------------------------

# Retinal Vascular Disease Detection using fundus image

--------------------------------------------------------------------------------
## Introduction
Diabetic Retinopathy (DR) is a serious eye condition that can lead to blindness
if not detected early. This project leverages deep learning techniques using InceptionV3 to classify fundus images into different DR severity levels. The model is trained on preprocessed fundus images to classify them into five categories:
*   No DR (Healthy Retina)
*   Mild NPDR (Non-Proliferative Diabetic Retinopathy)
*   Moderate NPDR
*   Severe NPDR
*   PDR (Proliferative Diabetic Retinopathy)

This solution provides an automated system that assists ophthalmologists in early detection and diagnosis of Diabetic Retinopathy.

--------------------------------------------------------------------------------

## Objective
The primary objective of this project is to develop a deep learning model using the InceptionV3 architecture to automatically classify retinal images into different stages of Diabetic Retinopathy with high accuracy.

*   Preprocess and augment the dataset for improved model generalization.
*   Implement Transfer Learning using InceptionV3.
*   Train and evaluate the deep learning model for accurate DR detection.
*   Deploy the model using Flask and Gradio for easy accessibility.


--------------------------------------------------------------------------------


## Aim of the Project

*   To train and fine-tune an InceptionV3-based model on a labeled dataset of
fundus images.
*   To evaluate the performance of the trained model on unseen test data.
*   To deploy the trained model using Flask and Gradio for real-time predictions.



### Importing Required Libraries
This block loads essential libraries for data processing, model building, augmentation, and evaluation.

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import glorot_uniform,he_uniform
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1_l2

### Defining Dataset Paths
This section sets the file paths for training, validation, and testing datasets.

In [ ]:
# File paths
train_dir = "C:/Users/Nasir/Downloads/OUR/OUR/TRAIN"
val_dir = "C:/Users/Nasir/Downloads/OUR/OUR/VALIDATION"
test_dir = "C:/Users/Nasir/Downloads/OUR/OUR/TEST"

### Loading and Preprocessing Data

*   Loads images from the dataset directories.
*   Resizes them to 299x299 (required for InceptionV3).
*   Normalizes pixel values to [0,1] range.
*   Assigns labels based on folder names.






In [ ]:
# Load and preprocess data (same as before)
def load_data(directory):
    images = []
    labels = []
    label_dict = {
        'nodr': 0,
        'mild_npdr': 1,
        'moderate_npdr': 2,
        'severe_npdr': 3,
        'pdr': 4
    }

    for label_name, label_index in label_dict.items():
        label_dir = os.path.join(directory, label_name)
        for img_name in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_name)
            img = image.load_img(img_path, target_size=(299, 299))  # Resize to InceptionV3's required size
            img_array = image.img_to_array(img) / 255.0  # Normalize
            images.append(img_array)
            labels.append(label_index)

    return np.array(images), np.array(labels)

### Loading Train, Validation, and Test Datasets
This loads the dataset using the load_data function.

In [ ]:
# Load the datasets
x_train, y_train = load_data(train_dir)
x_val, y_val = load_data(val_dir)
x_test, y_test = load_data(test_dir)

In [ ]:
# Convert labels to one-hot encoding
num_classes = len(np.unique(y_train))  # Total number of classes
y_train = to_categorical(y_train, num_classes=num_classes)
y_val = to_categorical(y_val, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

In [ ]:
input_dim = x_train.shape
input_dim

In [ ]:
y_train.shape,x_val.shape

In [ ]:
x_test.shape,y_val.shape,y_test.shape

Building the InceptionV3 Model
This defines a deep learning model with:


*   InceptionV3 as the base model
*   Fully connected layers with batch normalization, dropout, and activation functions.





In [ ]:
def create_model():
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
    base_model.trainable = False  # Freeze the base model initially

    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())  # Global average pooling to reduce dimensions
    model.add(Flatten())

    model.add(Dense(1024, activation='relu', kernel_initializer=he_uniform()))
    model.add(BatchNormalization())  # Batch normalization

    model.add(Dense(512, activation='relu', kernel_initializer=he_uniform()))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))  # Dropout layer with rate 0.2 (alternate)

    model.add(Dense(256, activation='relu', kernel_initializer=he_uniform()))
    model.add(BatchNormalization())

    model.add(Dense(num_classes, activation='softmax'))  # Final output layer for classification

    return model

In [ ]:
model = create_model()

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

### Data Augmentation for Training
This applies transformations like rotation, shifting, shearing, and flipping for robust training.

In [ ]:
# Data augmentation setup (put this before model.fit)
train_datagen = ImageDataGenerator(

    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

### Training the Model
Fits the model using augmented training data.

In [ ]:
#early_stopping = EarlyStopping(monitor='val_loss', patience=10, mode='min', verbose=1)callbacks=[early_stopping],

# Fit the model with fine-tuning
history = model.fit(
    train_datagen.flow(x_train, y_train, batch_size=128),  # Corrected flow syntax
    validation_data=(x_val, y_val),
    epochs=20,
    verbose=1
)

### Model Evaluation
Evaluates performance on test data.

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

### Loss Plotting
Plots training vs validation loss.

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.title("Train Vs Val Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xlim(0, 50)
plt.xticks(range(0, 51, 2))
plt.ylim(0, 70)
plt.yticks(range(0, 71, 4))
plt.legend(['Val_loss','loss'])
plt.show()

### Flask Implementation for Deployment
This Flask app serves the model for real-time predictions.

In [ ]:
from flask import Flask, request, jsonify
from tensorflow.keras.preprocessing import image

app = Flask(__name__)

# Load trained model
model = create_model()
model.load_weights("C:/Users/Nasir/Downloads/OUR/model_weights.h5")

# Class labels
class_labels = ['No DR', 'Mild NPDR', 'Moderate NPDR', 'Severe NPDR', 'PDR']

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['image']
    img = image.load_img(file, target_size=(299, 299))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions)

    return jsonify({'prediction': class_labels[class_idx]})

if __name__ == '__main__':
    app.run(debug=True)


### Gradio Implementation for Deployment

In [ ]:
import gradio as gr
from tensorflow.keras.preprocessing import image

model = create_model()
model.load_weights('"C:/Users/Nasir/Downloads/OUR/51_r1.jpg".h5')

def predict_image(img):
    img = img.resize((299, 299))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions)
    class_labels = ['No DR', 'Mild NPDR', 'Moderate NPDR', 'Severe NPDR', 'PDR']
    predicted_class = class_labels[class_idx]
    return f'Predicted class: {predicted_class}'

iface = gr.Interface(
    fn=predict_image,
    inputs=gr.inputs.Image(shape=(299, 299)),
    outputs="text",
    title="Diabetic Retinopathy Detection"
)

iface.launch()


# Conclusion
This project successfully implements Diabetic Retinopathy detection using deep learning. The trained InceptionV3 model classifies fundus images into different severity levels. The final model is deployed using Flask and Gradio, allowing users to upload retinal images and get predictions.